In [1]:
import pandas as pd
from sklearn.cluster import KMeans

In [2]:
data=pd.read_csv('Geospatial_Coordinates.csv')
df1=pd.DataFrame(data)
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [3]:
data_2=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df2=pd.DataFrame(data_2[0])
df2=df2[df2['Borough']!='Not assigned'].reset_index().drop('index', axis=1)
df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
print(df2.shape)

(103, 3)


In [5]:
df3=pd.merge(df2,df1,on='Postal Code')
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Get data of Toronto neighborhoods only 

In [9]:
toronto=df3[df3['Borough'].astype(str).str.contains('Toronto')].reset_index(drop=True)
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [17]:
from geopy.geocoders import Nominatim
address='Toronto, CA'
location=Nominatim(user_agent='ca_explorer').geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographic cordinate of Toronto is {},{}.'.format(latitude, longitude))

The geographic cordinate of Toronto is 43.6534817,-79.3839347.


In [22]:
import folium
map_toronto=folium.Map(location=[latitude,longitude], zoom_start=11)
for lat,lng,label in zip(toronto['Latitude'],toronto['Longitude'],toronto['Neighborhood']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7
    ).add_to(map_toronto)

map_toronto

In [23]:
import requests
from pandas.io.json import json_normalize

In [43]:
CLIENT_ID='XOG5WQ43GVCYRKZVLBFHTZTWGHNDJ0JS0FNZZZUTINAXL1JB'
CLIENT_SECRET='DLZLVRFA5RXVND4BB2JZ4O45ZTNTHKNEMTXQELECOR0MDO3E'
VERSION='20140715'

## Explore the first neighborhood

In [66]:
name=toronto.loc[0,'Neighborhood']
lat=toronto.loc[0,'Latitude']
lng=toronto.loc[0,'Longitude']
limit=100
radius=500
print('{} has cordinate {},{}.'.format(name, lat, lng))

url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    limit)
print(url) 

Regent Park, Harbourfront has cordinate 43.6542599,-79.3606359.
https://api.foursquare.com/v2/venues/explore?&client_id=XOG5WQ43GVCYRKZVLBFHTZTWGHNDJ0JS0FNZZZUTINAXL1JB&client_secret=DLZLVRFA5RXVND4BB2JZ4O45ZTNTHKNEMTXQELECOR0MDO3E&v=20140715&ll=43.6542599,-79.3606359&radius=500&limit=100


In [67]:
result=requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5f095c087d48a8229663b885'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'contact': {},
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'd

In [59]:
def get_categories(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [68]:
venues=result['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
venues[0:5]

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '54ea41ad498e9a11e9e13308',
   'name': 'Roselle Desserts',
   'contact': {},
   'location': {'address': '362 King St E',
    'crossStreet': 'Trinity St',
    'lat': 43.653446723052674,
    'lng': -79.3620167174383,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.653446723052674,
      'lng': -79.3620167174383}],
    'distance': 143,
    'postalCode': 'M5A 1K9',
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['362 King St E (Trinity St)',
     'Toronto ON M5A 1K9',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d16a941735',
     'name': 'Bakery',
     'pluralName': 'Bakeries',
     'shortName': 'Bakery',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
      'suffix': '.png'},
     'primary': True}],
   'verified'

In [69]:
filtered_columns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]
nearby_venues['venue.categories']=nearby_venues.apply(get_categories,axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Corktown Common,Park,43.655618,-79.356211


## Explore all neigborhoods

In [77]:
def get_nearby_venues(names,latitudes,longitudes):
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
        results=requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(name,
                            lat,
                            lng,
                             v['venue']['name'],
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name'])
                            for v in results
                            ])
    nearby_venues=pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns=['Neighborhood name',
                          'Neighborhood lat',
                          'Neighborhood lng',
                          'Venue name',
                          'Venue lat',
                          'Venue lng',
                          'Venue category']
    return(nearby_venues)

In [79]:
toronto_venues=get_nearby_venues(names=toronto['Neighborhood'],
                                latitudes=toronto['Latitude'],
                                longitudes=toronto['Longitude'])
toronto_venues

,Neighborhood name,Neighborhood lat,Neighborhood lng,Venue name,Venue lat,Venue lng,Venue category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
5,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
7,"Regent Park, Harbourfront",43.654260,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
8,"Regent Park, Harbourfront",43.654260,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
9,"Regent Park, Harbourfront",43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


## One Hot encoding

In [89]:
toronto_onehot=pd.get_dummies(toronto_venues[['Venue category']], prefix='', prefix_sep='')
toronto_onehot.shape

(1625, 232)

In [95]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood name'] 
# define a list of column names
cols = toronto_onehot.columns.tolist()
cols

# move the column name to the beggining
cols.insert(0, cols.pop(cols.index('Neighborhood')))
cols

#then use .reindex() function to reorder
toronto_onehot = toronto_onehot.reindex(columns= cols)

#check result
toronto_onehot.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
toronto_group=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_group.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.071429,0.214286,0.071429,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,Central Bay Street,0.014286,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.014286,0.0,0.0,0.014286,0.0,0.0


In [102]:
def return_common(row, num_top_venues):
    row_categories=row.iloc[1:].sort_values(ascending=False)
    return row_categories.index.values[0:num_top_venues]

In [108]:
import numpy as np
num_top_venues=5
indicator=['st','nd','rd']

columns=['Neighborhood']
for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} most common venues'.format((i+1),indicator[i]))
    except:
        columns.append('{}th most common venues'.format(i+1))
    
    venue_sorted=pd.DataFrame(columns=columns)
    venue_sorted['Neighborhood']=toronto_group['Neighborhood']

In [109]:
for i in np.arange(toronto_group.shape[0]):
    venue_sorted.iloc[i,1:]=return_common(toronto_group.iloc[i,:],num_top_venues)
venue_sorted.head()

,Neighborhood,1st most common venues,2nd most common venues,3rd most common venues,4th most common venues,5th most common venues
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden,Brewery,Burrito Place,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Harbor / Marina,Boat or Ferry,Coffee Shop,Plane
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant


## KMeans clustering

In [110]:
from sklearn.cluster import KMeans

In [113]:
k=5
toronto_cluster=toronto_group.drop('Neighborhood',axis=1)
kmeans=KMeans(n_clusters=k, random_state=0).fit(toronto_cluster)

In [116]:
venue_sorted.insert(0,'Cluster Labels',kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [118]:
toronto_merged=toronto.drop('Postal Code', axis=1)
toronto_merged=toronto_merged.join(venue_sorted.set_index('Neighborhood'),on='Neighborhood')
toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st most common venues,2nd most common venues,3rd most common venues,4th most common venues,5th most common venues
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Café
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Café
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Restaurant,Café,Coffee Shop,Cocktail Bar,American Restaurant
4,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Women's Store,Department Store


## Map of clusters

In [120]:
map_cluster=folium.Map(location=[latitude,longitude], zoom_start=11)

import matplotlib.cm as cm
import matplotlib.colors as colors

x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range (k)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

In [121]:
for lat, lng, nei, cluster in zip (toronto_merged['Latitude'], toronto_merged['Longitude'],toronto_merged['Neighborhood'],toronto_merged['Cluster Labels']):
    label=folium.Popup(str(nei)+'Cluster'+str(cluster), parse_html=False)
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color=rainbow[cluster-1],
                       fill=True,
                       fill_color=rainbow[cluster-1],
                       fill_opacity=0.7).add_to(map_cluster)
map_cluster
    

In [122]:
toronto_merged.loc[toronto_merged['Cluster Labels']==0,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,1st most common venues,2nd most common venues,3rd most common venues,4th most common venues,5th most common venues
0,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Breakfast Spot,Café
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Yoga Studio,Bar,Beer Bar
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Café
3,St. James Town,Restaurant,Café,Coffee Shop,Cocktail Bar,American Restaurant
4,The Beaches,Health Food Store,Trail,Pub,Women's Store,Department Store
5,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Restaurant
6,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant
7,Christie,Grocery Store,Café,Park,Nightclub,Diner
8,"Richmond, Adelaide, King",Coffee Shop,Café,Hotel,Restaurant,Gym
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Pet Store,Middle Eastern Restaurant,Music Venue


## Examine clustering

In [123]:
toronto_merged.loc[toronto_merged['Cluster Labels']==1,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,1st most common venues,2nd most common venues,3rd most common venues,4th most common venues,5th most common venues
18,Lawrence Park,Park,Bus Line,Swim School,Women's Store,Dessert Shop


In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels']==2,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,1st most common venues,2nd most common venues,3rd most common venues,4th most common venues,5th most common venues
19,Roselawn,Garden,Home Service,Department Store,Falafel Restaurant,Event Space


In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels']==3,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,1st most common venues,2nd most common venues,3rd most common venues,4th most common venues,5th most common venues
21,"Forest Hill North & West, Forest Hill Road Park",Trail,Mexican Restaurant,Jewelry Store,Sushi Restaurant,Women's Store


In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels']==4,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Neighborhood,1st most common venues,2nd most common venues,3rd most common venues,4th most common venues,5th most common venues
29,"Moore Park, Summerhill East",Park,Lawyer,Trail,Restaurant,Deli / Bodega
33,Rosedale,Park,Playground,Trail,Deli / Bodega,Event Space
